In [1]:
%load_ext autoreload                                                                          
%autoreload 2  
%env METAPACK_CACHE /Volumes/Cache/metapack
%env ROWGEN_CACHE /Volumes/Cache/metapack
    
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

%matplotlib inline

sns.set_context('notebook')
mp.jupyter.init()

# Temp, b/c I don't want to research jupyter to get the value from .zshrc

from rowgenerators import get_cache
from rowgenerators.appurl.util import get_cache_name 
get_cache_name(None), get_cache()

env: METAPACK_CACHE=/Volumes/Cache/metapack
env: ROWGEN_CACHE=/Volumes/Cache/metapack


('metapack', OSFS('/Volumes/Cache/metapack'))

In [2]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()

pkg.set_sys_path()
from pylib import *

pkg

# Select tables from the Credential Engine Registry
`credentialengine.org-registry-1.1.1` Last Update: 2023-10-18T22:36:10

_Tables of data extracted from the Credential Engin JSON-LD file_




 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)


## References
<ul><li> <strong><a href="http://ds.civicknowledge.org.s3.amazonaws.com/credentialengine.org/ce_registry_1697667117_ddfe24e4dcbf402e2cb741986b6676fc.zip">registry_source</a></strong>. </li><ul>

In [3]:
s = pkg.reference('registry_source').resolved_url
print(type(s), s)
r = s.get_resource()
r

<class 'rowgenerators.appurl.web.web.WebUrl'> http://ds.civicknowledge.org.s3.amazonaws.com/credentialengine.org/ce_registry_1697667117_ddfe24e4dcbf402e2cb741986b6676fc.zip


<ZipUrl file:///Volumes/Cache/metapack/ds.civicknowledge.org.s3.amazonaws.com/credentialengine.org/ce_registry_1697667117_ddfe24e4dcbf402e2cb741986b6676fc.zip>

In [10]:
r.fspath

PosixPath('/Volumes/Cache/metapack/ds.civicknowledge.org.s3.amazonaws.com/credentialengine.org/ce_registry_1697667117_ddfe24e4dcbf402e2cb741986b6676fc.zip')

In [63]:
import json

def visit_nodes(json_obj, path="", json_path="", results=None):
    if results is None:
        results = []
    if isinstance(json_obj, dict):
        for k, v in json_obj.items():
            new_path = f"{path}.{k}" if path else k
            new_json_path = f"{json_path}['{k}']" if json_path else f"['{k}']"
            if not isinstance(v, (dict, list)):
                results.append((new_path, v, new_json_path))
            visit_nodes(v, new_path, new_json_path, results)
    elif isinstance(json_obj, list):
        for i, item in enumerate(json_obj):
            new_path = f"{path}[{i}]"
            new_json_path = f"{json_path}[{i}]"
            if not isinstance(item, (dict, list)):
                results.append((new_path, item, new_json_path))
            visit_nodes(item, new_path, new_json_path, results)
    return results

def getIPT(k, v, p, d):
    if 'instructionalProgramType' in k and k.endswith('@type'):
        p = p.replace("['@type']",'')
        return eval(f"d{p}")
    else:
        return None

def get_en_str(v):

    if isinstance(v, str):
        return v

    for k in ('en', 'en-US', 'en-AU', 'en-GB' ):
        
        try:
            return v[k]
        except KeyError:
            pass
    
        try:
            return v[k.lower()]
        except KeyError:  
            pass
    
    raise KeyError()
    
    

def extract_fields(d):
    
    try:
        d['decoded_resource']['@graph']
    except KeyError:
        return 
    
    for e in d['decoded_resource']['@graph']:

        try:
            type_ = e['@type']
            desc = get_en_str(e['ceterms:description'])
            name = get_en_str(e['ceterms:name'])
            ctid = e['ceterms:ctid']
        except KeyError:
            continue 
        
        for k, v, p in visit_nodes(e):
            ipt = getIPT(k, v, p, e)
            if ipt:
                try:
                    yield {
                        'ctid': ctid,
                        'name': name,
                        'description': desc,
                        'cred_type': type_.replace('ceterms:',''),
                        'cip': ipt['ceterms:codedNotation']
                    }
                except KeyError:
                    continue

import json
from tqdm.auto import tqdm
from pathlib import Path 
from itertools import islice

recs = []

#for e in tqdm(all_files):   
#    fn = e.get_target().fspath

# The +'_d' nonsense only works after the zip file has been expanded and rename,
# or it's been iterated through once

for fn in tqdm(Path(str(r.fspath)+'_d').glob('**/*.json')):
    t = fn.read_text()
    d = json.loads(t)
    recs.extend(list(extract_fields(d)))

df = pd.DataFrame(recs)
df.sample(20)

0it [00:00, ?it/s]

,ctid,name,description,cred_type,cip
14030,ce-4595245f-e7f9-46a1-b7ae-898887ba1214,Health Information Management,Health Information Management,AssociateDegree,51.0707
7305,ce-5d3d1bfa-a006-4998-b316-ed0064bf948a,Master of Public Affairs,The Master of Public Affairs (MPA) degree requ...,MasterDegree,44.0401
35253,ce-3efd3a3d-d9c2-4fd6-a3e3-bc34d3fe6214,Intuit QuickBooks Certified User (QBCU),This course will give you hands-on experience ...,Certification,22.0302
35531,ce-3bbff02f-5494-4831-b85c-07e800795300,Paralegal - Certificate,Graduates of this program know the principles ...,LearningOpportunityProfile,22.0302
15705,ce-5f9e1dec-e071-48ca-bb6f-bbb565d69fed,Certificate of Achievement in ESL Milestones -...,The ESL Milestone - Pathway to Environmental S...,Certificate,32.0108
3608,ce-9360c362-5b6d-408d-bc49-ebb081bb8bdb,Certificate in Machine Tool Technology,The Machine Tool Technology program offers bot...,Certificate,48.0503
36533,ce-c1c4f881-730e-4ae4-8aac-726b8c23b7c7,Teacher Certification in English as a Second L...,Teacher Certification in English as a Second L...,Certification,13.1401
11073,ce-7f908028-3a03-4411-b11b-75d9d16ae2e0,Forensic Investigation,A program that focuses on the systematic study...,MasterDegree,45.0401
23162,ce-d44df978-609e-4e13-9e8f-1ce426e598ee,Computer Science Web Developer,Information Technology has become an integral ...,Certificate,11.0101
43116,ce-f9b9b7fa-1ad3-429c-be13-55932c46d8a6,"PMP, Six Sigma, MS Project, MySQL Program",For those with experience managing or coordina...,LearningOpportunityProfile,11.0101
